In [43]:
import pandas as pd

In [44]:
training_data= pd.read_csv("X_Y_train.csv")
testing_data= pd.read_csv("X_test.csv")


In [76]:
training_data.shape

(10980, 12)

In [46]:
testing_data.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


In [47]:
x_train= training_data.iloc[:,7]
y_train= training_data.iloc[:,1]
x_test= testing_data.iloc[:,6]

In [48]:
def y_transform(y):
    if(y=="positive"):
        return 0
    elif(y=="neutral"):
        return 1
    else:
        return 2

In [49]:
y_train= y_train.apply(y_transform)

In [50]:
y_train

0        2
1        0
2        0
3        2
4        2
        ..
10975    1
10976    0
10977    2
10978    2
10979    2
Name: airline_sentiment, Length: 10980, dtype: int64

In [51]:
from nltk.tokenize import word_tokenize

In [52]:
def forming_words(string):
    return word_tokenize(string)

In [53]:
x_train= x_train.apply(forming_words)
x_test= x_test.apply(forming_words)

In [54]:
from nltk.stem import WordNetLemmatizer
leammatizer= WordNetLemmatizer()

In [55]:
from nltk.corpus import wordnet

def get_simple_pos(tag):
    
    if(tag.startswith('J')):
        return wordnet.ADJ
    elif(tag.startswith('V')):
        return wordnet.VERB
    elif(tag.startswith('N')):
        return wordnet.NOUN
    elif(tag.startswith('R')):
        return wordnet.ADJ
    else:
        return wordnet.NOUN

In [56]:
from nltk import pos_tag

In [57]:
from nltk.corpus import stopwords
import string

stops= set(stopwords.words('english'))
punctuations= list(string.punctuation)
stops.update(punctuations)

In [58]:
def clean_review(words):
    output_words=[]
    
    for w in words:
        if(w.lower() not in stops):
            pos= pos_tag([w])
            clean_word = leammatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [59]:
x_train= x_train.apply(clean_review)
x_test= x_test.apply(clean_review)

In [60]:
def merging_words(words):
    return " ".join(words)

In [61]:
x_train= x_train.apply(merging_words)
x_test= x_test.apply(merging_words)

In [62]:
x_train

0        southwestair schedule morning 2 day fact yes.....
1        southwestair see worker time time go beyond lo...
2        united flew ord miami back great crew service ...
3                 southwestair dultch97 's horse radish 😤🐴
4        united flight ord delayed air force one last f...
                               ...                        
10975                               americanair followback
10976    united thanks help wish phone rep could accomi...
10977             usairways worst ever dca customerservice
10978         nrhodes85 look another apology fly usairways
10979    united far bad airline 4 plane delay 1 round t...
Name: text, Length: 10980, dtype: object

In [63]:
from sklearn.feature_extraction.text import CountVectorizer

In [64]:
count_vec= CountVectorizer(max_features=8370,ngram_range=(1,2))

In [65]:
x_train_features = count_vec.fit_transform(x_train)

In [66]:
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [67]:
y_train.shape

(10980,)

In [68]:
len(count_vec.get_feature_names())

8370

In [69]:
x_test_features= count_vec.transform(x_test)

In [70]:
x_test_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [71]:
from sklearn.naive_bayes import MultinomialNB
clf_naive_bayes= MultinomialNB()
clf_naive_bayes.fit(x_train_features,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [72]:
y_pred = clf_naive_bayes.predict(x_test_features)


In [73]:
import numpy as np

In [74]:
np.savetxt("y_pred_twitter.csv",y_pred)

In [75]:
y_pred

array([2, 2, 2, ..., 1, 0, 1], dtype=int64)

In [33]:
y_pred_word = []
for i in y_pred:
    if(i==0):
        y_pred_word.append("positive")
    elif(i==1):
        y_pred_word.append("neutral")
    else:
        y_pred_word.append("negative")

In [34]:
y_pred_word

['negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negat

In [35]:
np.savetxt("y_pred_twitter_wording.csv",y_pred_word,delimiter=',',fmt="%s")